## Multi-task ElasticNet

Es una técnica de regresión multivariada que extiende el algoritmo ElasticNet para resolver problemas en los que hay múltiples tareas o salidas. La principal diferencia con ElasticNet tradicional es que, en lugar de predecir una sola variable objetivo, se predicen varias variables objetivo (también llamadas "tareas"), y todas comparten la misma estructura regularizadora.

¿Cómo funciona?
ElasticNet combina dos tipos de regularización:

L1 (Lasso): promueve la esparsidad, es decir, trata de hacer que algunos coeficientes del modelo sean cero, eliminando características innecesarias.
L2 (Ridge): favorece la reducción de los coeficientes para que el modelo no sobreajuste los datos.
MultiTaskElasticNet aplica esta combinación de regularización a todas las tareas de manera conjunta, promoviendo relaciones entre las tareas a la vez que busca minimizar el error en las predicciones de todas ellas.

Características:
Común a todas las tareas: Las regularizaciones L1 y L2 se comparten entre todas las tareas.
Tareas relacionadas: Se asume que las tareas son relacionadas y pueden beneficiarse mutuamente, lo cual se logra gracias a la regularización conjunta.

Formula:

$$
 \text{min} \left( \sum_{i=1}^{N} \left( \| Y_i - X\beta_i \|^2 \right) + \lambda \left( \alpha \| \beta_i \|_1 + \frac{1-\alpha}{2} \| \beta_i \|_2^2 \right) \right) 
$$

Donde:
- ${Y_i}$ es el vector de valores observados para la ${i}$-ésima tarea.
- ${X}$ es la matriz de características.
- ${\beta_i}$ es el vector de coeficientes para la ${i}$-ésima tarea.
- ${\lambda}$ controla la fuerza de la regularización.
- ${\alpha}$ es un parámetro que controla la mezcla de L1 y L2 (0 corresponde solo a Ridge, 1 corresponde solo a Lasso).

In [1]:
import plotly.graph_objects as go
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Crear un conjunto de datos con múltiples salidas
X, Y = make_regression(n_samples=1000, n_features=10, n_targets=3, noise=0.5, random_state=42)

# Dividir los datos en entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 10)
(200, 10)
(800, 3)
(200, 3)


In [2]:
# crear modelo de regresion lineal simple con el conjunto de entrenamiento
regression = MultiTaskElasticNet(alpha=0.1, l1_ratio=0.5)
regression.fit(x_train,y_train)
regression

MultiTaskElasticNet(alpha=0.1)

In [3]:
# Predecir el conjunto de testeo, solo toma los valores x para predecir los valores de y_test
y_pred = regression.predict(x_test)
print(f"Mean squared error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"""Max {y_test.max()}|Min {y_test.min()}|Mean {y_test.mean()}\nMax {y_pred.max()}|Min {y_pred.min()}|Mean {y_pred.mean()}""")
print(regression.coef_)
print(f"Coeficiente de determinación (R^2): {regression.score(x_test, y_test):.2f}")
# y_pred[:,0]

Mean squared error: 79.15
Max 596.8287279453774|Min -526.1551298652129|Mean 12.972312048870508
Max 566.8870156060781|Min -500.4701226652846|Mean 12.021525589320039
[[49.45846332 69.17966915  9.17945262 30.41374631 19.95260006 40.47360033
  60.43597775 72.70885872 32.02860362 58.11280697]
 [78.1051163  71.09892023 55.58967831  9.88463471 49.76110432 56.5399178
  15.98645465 70.96453094 71.26385358 14.41577843]
 [68.99424359 37.75245236  5.22217599  6.48713907 87.93172983 69.7356741
  64.29798868 23.71795919 28.07191571 78.84733163]]
Coeficiente de determinación (R^2): 1.00


In [4]:
fig = go.Figure([
    go.Scatter(x=x_train[:,0].squeeze(), y=y_train[:,0], name='Train', mode='markers'),
    go.Scatter(x=x_test[:,0].squeeze(), y=y_test[:,0], name='Test', mode='markers'
                ,marker=dict(size=8)),
    go.Scatter(x=x_test[:,0].squeeze(), y=y_pred[:,0], name='Prediction', mode='markers',
                marker=dict(size=4))
])
fig.update_layout(title='Diabetes dataset', xaxis_title='X', yaxis_title='Y', template='plotly_dark')
fig.show()